In [ ]:
import pandas as pd
pd.set_option("display.max_columns", None)

In [ ]:
df_itens.isnull().sum()/df_itens.shape[0]*100

ID_ITEM                     0.0
ID_IDTIPOFT2                0.0
ID_IDFAMILIA                0.0
ST_ESTADOFT_DETEC           0.0
TX_TEXTOESTADOFT_DETEC      0.0
                           ... 
VL_AREALIQUIDAPECA          0.0
VL_AREABRUTACHAPA           0.0
VL_AREALIQUIDACHAPA         0.0
VL_VOLUMEPALETEFECHADOM3    0.0
VL_VOLUMEPACOTEFECHADOM3    0.0
Length: 81, dtype: float64

### Inicio

In [ ]:
import pandas as pd
import numpy as np
from pathlib import Path
df_maquina = pd.read_parquet('../../data/02 - trusted/parquet/typed_tb_maquina.parquet')
df_facas = pd.read_parquet('../../data/02 - trusted/parquet/typed_tb_facas.parquet')
df_paradas = pd.read_parquet('../../data/02 - trusted/parquet/typed_tb_paradas.parquet')
df_clientes = pd.read_parquet('../../data/02 - trusted/parquet/typed_tb_clientes.parquet')
df_pedidos = pd.read_parquet('../../data/02 - trusted/parquet/typed_tb_pedidos.parquet')
df_itens = pd.read_parquet('../../data/02 - trusted/parquet/typed_tb_itens.parquet')

In [ ]:
import sys

sys.path.append("./../../")
from src.data.data_treatment import corrigir_tarefcon_relacoes
df_tarefcon = pd.read_parquet('../../data/02 - trusted/parquet/typed_tb_tarefcon.parquet')
df_tarefcon = corrigir_tarefcon_relacoes(df_tarefcon)

In [ ]:
print(f"TAREFCON: {df_tarefcon.shape}")
print(f"PEDIDOS: {df_pedidos.shape}")
print(f"ITENS: {df_itens.shape}")
print(f"MAQUINA: {df_maquina.shape}")
print(f"FACAS: {df_facas.shape}")
print(f"PARADAS: {df_paradas.shape}")
print(f"CLIENTES: {df_clientes.shape}")

TAREFCON: (1604562, 26)
PEDIDOS: (384533, 92)
ITENS: (69774, 81)
MAQUINA: (50, 4)
FACAS: (8139, 4)
PARADAS: (699, 5)
CLIENTES: (5586, 4)


In [ ]:
def identificar_colunas_comuns(df1, df2, nome1, nome2):
    """Identifica colunas que existem em ambas tabelas"""
    comuns = set(df1.columns) & set(df2.columns)

    if comuns:
        print(f"\n Colunas comuns entre {nome1} e {nome2}:")
        for col in sorted(comuns):
            print(f"   - {col}")
    print(f"Lista de colunas comuns entre {nome1} e {nome2}:", list(comuns))
    return list(comuns)

In [ ]:
comuns_pedidos = identificar_colunas_comuns(df_tarefcon, df_pedidos, "TAREFCON", "PEDIDOS")


 Colunas comuns entre TAREFCON e PEDIDOS:
   - ID_IDCLIENTE
   - ID_ITEM
   - ID_PEDIDO
   - VL_ARRANJO
   - VL_GRAMATURA
Lista de colunas comuns entre TAREFCON e PEDIDOS: ['VL_ARRANJO', 'ID_PEDIDO', 'ID_ITEM', 'ID_IDCLIENTE', 'VL_GRAMATURA']


['VL_ARRANJO', 'ID_PEDIDO', 'ID_ITEM', 'ID_IDCLIENTE', 'VL_GRAMATURA']

In [ ]:
comuns_itens = identificar_colunas_comuns(df_tarefcon, df_itens, "TAREFCON", "ITENS")


 Colunas comuns entre TAREFCON e ITENS:
   - ID_IDCLIENTE
   - ID_ITEM
   - VL_GRAMATURA
Lista de colunas comuns entre TAREFCON e ITENS: ['ID_ITEM', 'ID_IDCLIENTE', 'VL_GRAMATURA']


In [ ]:
comuns_itens_x_pedidos = identificar_colunas_comuns(df_pedidos, df_itens, "PEDIDOS", "ITENS")


 Colunas comuns entre PEDIDOS e ITENS:
   - CD_CODIGOREFERENCIA
   - CD_COMPOSICAO
   - CD_TIPOABNT
   - FL_ESPELHO
   - FL_EXIGELAUDO
   - FL_FILME
   - FL_LAPINTERNO
   - FL_LAPNOCOMP
   - FL_PALETIZADO
   - FL_REFILADO
   - FL_RESINAINTERNA
   - ID_FACA
   - ID_IDCLIENTE
   - ID_IDPALETE
   - ID_IDTIPOFT2
   - ID_ITEM
   - QT_NRCORES
   - QT_PACOTESPORPALETE
   - QT_PECASPORPACOTE
   - QT_PECASPORPALETE
   - QT_UNIDADESPORPALETE
   - TX_COR1
   - TX_COR2
   - TX_COR3
   - TX_COR4
   - TX_PATHFIGURADOLASTRO
   - TX_REFERENCIA
   - VL_ALTURAINTERNA
   - VL_ALTURAPACOTE
   - VL_ALTURAPALETEFECHADO
   - VL_AREABRUTACHAPA
   - VL_AREABRUTAPECA
   - VL_AREABRUTAPECACOMREFILOS
   - VL_AREALIQUIDACHAPA
   - VL_AREALIQUIDAPECA
   - VL_COBBINTMAXIMO
   - VL_COLUNAMINIMO
   - VL_COMPPACOTE
   - VL_COMPPECA
   - VL_COMPRESSAO
   - VL_COMPRIMENTO
   - VL_COMPRIMENTOINTERNO
   - VL_CONSUMOCOR1
   - VL_CONSUMOCOR2
   - VL_CONSUMOCOR3
   - VL_CONSUMOCOR4
   - VL_GRAMATURA
   - VL_LAP
   - VL_LARGP

In [ ]:
df_pedidos_prep = df_pedidos.copy()
rename_pedidos = {}

campos_renomear = ['VL_GRAMATURA', 'VL_ARRANJO']
for campo in campos_renomear:
    if campo in df_pedidos_prep.columns:
        rename_pedidos[campo] = f"{campo}_PEDIDO"


if rename_pedidos:
    df_pedidos_prep = df_pedidos_prep.rename(columns=rename_pedidos)
    print(f"PEDIDOS - Campos renomeados: {list(rename_pedidos.keys())}")


#  ITENS - Adicionar sufixo
df_itens_prep = df_itens.copy()
rename_itens = {}

if 'ID_TIPOFT2' in df_itens_prep.columns:
    rename_itens['ID_TIPOFT2'] = 'ID_TIPOFT2_ITEM'

if rename_itens:
    df_itens_prep = df_itens_prep.rename(columns=rename_itens)
    print(f"ITENS - Campos renomeados: {list(rename_itens.keys())}")
# MAQUINA - Renomear TIPO
df_maquina_prep = df_maquina.copy()
if 'CD_TIPO' in df_maquina_prep.columns:
    df_maquina_prep = df_maquina_prep.rename(columns={'CD_TIPO': 'CD_TIPO_MAQUINA'})
    print("MAQUINA - CD_TIPO → CD_TIPO_MAQUINA")
elif 'ID_TIPO' in df_maquina_prep.columns:
    df_maquina_prep = df_maquina_prep.rename(columns={'ID_TIPO': 'ID_TIPO_MAQUINA'})
    print("MAQUINA - ID_TIPO → ID_TIPO_MAQUINA")

# FACAS - Renomear campos
df_facas_prep = df_facas.copy()
rename_facas = {}
if 'ST_STATUS' in df_facas_prep.columns:
    rename_facas['ST_STATUS'] = 'ST_STATUS_FACA'
if 'VL_COMPLAMINA' in df_facas_prep.columns:
    rename_facas['VL_COMPLAMINA'] = 'VL_COMPRIMENTO_LAMINA'

if rename_facas:
    df_facas_prep = df_facas_prep.rename(columns=rename_facas)
    print(f"FACAS - Campos renomeados: {list(rename_facas.keys())}")


# PARADAS
df_paradas_prep = df_paradas.copy()
rename_paradas = {}
if 'TX_DESCRICAO' in df_paradas_prep.columns:
    rename_paradas['TX_DESCRICAO'] = 'TX_DESCRICAO_PARADA'
if 'FL_USADACONVERSAO' in df_paradas_prep.columns:
    rename_paradas['FL_USADACONVERSAO'] = 'FL_PARADA_CONVERSAO'

if rename_paradas:
    df_paradas_prep = df_paradas_prep.rename(columns=rename_paradas)
    print(f"PARADAS - Campos renomeados: {list(rename_paradas.keys())}")

# CLIENTES
df_clientes_prep = df_clientes.copy()



PEDIDOS - Campos renomeados: ['VL_GRAMATURA', 'VL_ARRANJO']
MAQUINA - CD_TIPO → CD_TIPO_MAQUINA
FACAS - Campos renomeados: ['ST_STATUS', 'VL_COMPLAMINA']
PARADAS - Campos renomeados: ['TX_DESCRICAO', 'FL_USADACONVERSAO']

Todas as tabelas preparadas


In [ ]:
import pandas as pd
import numpy as np

def otimizar_memoria_df(df):
    """
    Reduz uso de memória do dataframe otimizando tipos de dados
    """
    print(f"\nMemória antes: {df.memory_usage(deep=True).sum() / 1024**2:.2f} MB")
    
    for col in df.columns:
        col_type = df[col].dtype
        
        # Otimizar tipos numéricos
        if col_type == 'float64':
            # Tentar converter para float32 se não houver perda de precisão
            if df[col].max() < 3.4e38 and df[col].min() > -3.4e38:
                df[col] = df[col].astype('float32')
        
        elif col_type == 'int64':
            # Converter para menor tipo inteiro possível
            c_min = df[col].min()
            c_max = df[col].max()
            
            if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                df[col] = df[col].astype('int8')
            elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                df[col] = df[col].astype('int16')
            elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                df[col] = df[col].astype('int32')
        
        # Otimizar objetos (strings)
        elif col_type == 'object':
            num_unique = df[col].nunique()
            num_total = len(df[col])
            
            # Se tem poucos valores únicos, converter para category
            if num_unique / num_total < 0.5:  # Menos de 50% valores únicos
                df[col] = df[col].astype('category')
    
    print(f"Memória depois: {df.memory_usage(deep=True).sum() / 1024**2:.2f} MB")
    print(f"Redução: {(1 - df.memory_usage(deep=True).sum() / df.memory_usage(deep=True).sum()) * 100:.1f}%")
    
    return df

# Aplicar a TODOS os dataframes ANTES do merge
print("Otimizando TAREFCON...")
df_tarefcon = otimizar_memoria_df(df_tarefcon)

print("\nOtimizando PEDIDOS...")
df_pedidos_prep = otimizar_memoria_df(df_pedidos_prep)

print("\nOtimizando ITENS...")
df_itens_prep = otimizar_memoria_df(df_itens_prep)



Otimizando TAREFCON...

Memória antes: 1291.24 MB
Memória depois: 218.46 MB
Redução: 0.0%

Otimizando PEDIDOS...

Memória antes: 668.37 MB
Memória depois: 154.54 MB
Redução: 0.0%

Otimizando ITENS...

Memória antes: 110.03 MB
Memória depois: 31.00 MB
Redução: 0.0%


### Tarefcon

In [ ]:
print(f"Registros originais: {len(df_tarefcon):,}")

# Identificar nomes corretos das colunas em TAREFCON
col_inicio = 'DT_INICIO' if 'DT_INICIO' in df_tarefcon.columns else 'INICIO'
col_duracao = 'VL_DURACAO' if 'VL_DURACAO' in df_tarefcon.columns else 'DURACAO'

# Aplicar filtros
df_tarefcon_filtrado = df_tarefcon[
    (df_tarefcon[col_inicio].notna()) &
    (df_tarefcon[col_duracao].notna()) &
    (df_tarefcon[col_duracao] > 0)
].copy()

print(f"Após filtros: {len(df_tarefcon_filtrado):,}")
print(f"Removidos: {len(df_tarefcon) - len(df_tarefcon_filtrado):,}")


Registros originais: 1,604,562
Após filtros: 1,604,242
Removidos: 320


### explorar diferenças entre df_item e df_pedidos

In [ ]:
# 1. Verificar se ITEM existe em ambas
itens_em_pedidos = set(df_pedidos['ID_ITEM'].unique())
itens_em_cadastro = set(df_itens['ID_ITEM'].unique())

print(f"Itens únicos em PEDIDOS: {len(itens_em_pedidos):,}")
print(f"Itens únicos em ITENS: {len(itens_em_cadastro):,}")
print(f"Itens em PEDIDOS mas não em ITENS: {len(itens_em_pedidos - itens_em_cadastro):,}")
print(f"Itens em ITENS mas nunca vendidos: {len(itens_em_cadastro - itens_em_pedidos):,}")


Itens únicos em PEDIDOS: 29,740
Itens únicos em ITENS: 69,774
Itens em PEDIDOS mas não em ITENS: 21,613
Itens em ITENS mas nunca vendidos: 61,647


In [ ]:
# Obter colunas de cada tabela (excluindo prefixos se necessário)
colunas_itens = set(df_itens.columns)
colunas_pedidos = set(df_pedidos.columns)

# Identificar colunas comuns (nomes exatos)
colunas_comuns_exatas = colunas_itens & colunas_pedidos

print(f"\nColunas em ITENS: {len(colunas_itens)}")
print(f"Colunas em PEDIDOS: {len(colunas_pedidos)}")
print(f"Colunas comuns (nome exato): {len(colunas_comuns_exatas)}")

if colunas_comuns_exatas:
    print("\nColunas comuns encontradas:")
    for col in sorted(colunas_comuns_exatas):
        print(f"  - {col}")

# ============================================================
# PARTE 2: IDENTIFICAR COLUNAS SEMANTICAMENTE EQUIVALENTES
# ============================================================

# Muitas vezes as colunas têm nomes ligeiramente diferentes
# mas representam o mesmo conceito

print("\n" + "=" * 80)
print("IDENTIFICAÇÃO DE COLUNAS SEMANTICAMENTE EQUIVALENTES")
print("=" * 80)

# Função para extrair o "nome base" da coluna (sem prefixo)
def extrair_nome_base(col):
    """Remove prefixos tipo VL_, ID_, FL_, etc"""
    if '_' in col:
        return col.split('_', 1)[1]  # Remove primeiro segmento
    return col

# Criar mapeamento
itens_base = {extrair_nome_base(col): col for col in colunas_itens}
pedidos_base = {extrair_nome_base(col): col for col in colunas_pedidos}

# Encontrar bases comuns
bases_comuns = set(itens_base.keys()) & set(pedidos_base.keys())

print(f"\nColunas semanticamente equivalentes: {len(bases_comuns)}")

# Criar dicionário de mapeamento
mapeamento = {}
for base in sorted(bases_comuns):
    col_itens = itens_base[base]
    col_pedidos = pedidos_base[base]
    mapeamento[base] = {
        'ITENS': col_itens,
        'PEDIDOS': col_pedidos
    }
    print(f"  {base:30s}: {col_itens:40s} ↔ {col_pedidos:40s}")

# ============================================================
# PARTE 3: PREPARAR DADOS PARA COMPARAÇÃO
# ============================================================

print("\n" + "=" * 80)
print("PREPARAÇÃO DOS DADOS PARA COMPARAÇÃO")
print("=" * 80)

# Identificar chave de junção
# A chave deve ser ID_ITEM ou ITEM
chave_item_itens = 'ID_ITEM' if 'ID_ITEM' in df_itens.columns else 'ITEM'
chave_item_pedidos = 'ID_ITEM' if 'ID_ITEM' in df_pedidos.columns else 'ITEM'

print(f"\nChave de junção:")
print(f"  ITENS: {chave_item_itens}")
print(f"  PEDIDOS: {chave_item_pedidos}")

# Selecionar apenas colunas relevantes de cada tabela
colunas_itens_comparar = [chave_item_itens] + [mapeamento[base]['ITENS'] for base in bases_comuns]
colunas_pedidos_comparar = [chave_item_pedidos, 'ID_PEDIDO'] + [mapeamento[base]['PEDIDOS'] for base in bases_comuns]

# Filtrar colunas que existem
colunas_itens_comparar = [c for c in colunas_itens_comparar if c in df_itens.columns]
colunas_pedidos_comparar = [c for c in colunas_pedidos_comparar if c in df_pedidos.columns]

df_itens_comp = df_itens[colunas_itens_comparar].copy()
df_pedidos_comp = df_pedidos[colunas_pedidos_comparar].copy()



Colunas em ITENS: 81
Colunas em PEDIDOS: 92
Colunas comuns (nome exato): 72

Colunas comuns encontradas:
  - CD_CODIGOREFERENCIA
  - CD_COMPOSICAO
  - CD_TIPOABNT
  - FL_ESPELHO
  - FL_EXIGELAUDO
  - FL_FILME
  - FL_LAPINTERNO
  - FL_LAPNOCOMP
  - FL_PALETIZADO
  - FL_REFILADO
  - FL_RESINAINTERNA
  - ID_FACA
  - ID_IDCLIENTE
  - ID_IDPALETE
  - ID_IDTIPOFT2
  - ID_ITEM
  - QT_NRCORES
  - QT_PACOTESPORPALETE
  - QT_PECASPORPACOTE
  - QT_PECASPORPALETE
  - QT_UNIDADESPORPALETE
  - TX_COR1
  - TX_COR2
  - TX_COR3
  - TX_COR4
  - TX_PATHFIGURADOLASTRO
  - TX_REFERENCIA
  - VL_ALTURAINTERNA
  - VL_ALTURAPACOTE
  - VL_ALTURAPALETEFECHADO
  - VL_AREABRUTACHAPA
  - VL_AREABRUTAPECA
  - VL_AREABRUTAPECACOMREFILOS
  - VL_AREALIQUIDACHAPA
  - VL_AREALIQUIDAPECA
  - VL_COBBINTMAXIMO
  - VL_COLUNAMINIMO
  - VL_COMPPACOTE
  - VL_COMPPECA
  - VL_COMPRESSAO
  - VL_COMPRIMENTO
  - VL_COMPRIMENTOINTERNO
  - VL_CONSUMOCOR1
  - VL_CONSUMOCOR2
  - VL_CONSUMOCOR3
  - VL_CONSUMOCOR4
  - VL_GRAMATURA
  - VL

In [ ]:
df_pedidos_limpo = df_pedidos_comp.copy()

# Remover duplicatas de colunas se houver
df_pedidos_limpo = df_pedidos_limpo.loc[:, ~df_pedidos_limpo.columns.duplicated()]


In [ ]:
colunas_chave = ['ID_ITEM', 'ITEM', 'ID_PEDIDO', 'PEDIDO']
colunas_renomear = [col for col in df_pedidos_limpo.columns if col not in colunas_chave]

# Adicionar sufixo _PEDIDO a todas as colunas que não são chave
rename_dict_seguro = {col: f"{col}_PEDIDO" for col in colunas_renomear}


In [ ]:
df_pedidos_limpo = df_pedidos_limpo.rename(columns=rename_dict_seguro)


In [ ]:
df_itens_limpo = df_itens_comp.copy()

# Remover duplicatas de colunas se houver
df_itens_limpo = df_itens_limpo.loc[:, ~df_itens_limpo.columns.duplicated()]

# Adicionar sufixo _ITEM a todas as colunas que não são ID_ITEM
colunas_renomear_itens = [col for col in df_itens_limpo.columns if col != chave_item_itens]
rename_dict_itens = {col: f"{col}_ITEM" for col in colunas_renomear_itens}

df_itens_limpo = df_itens_limpo.rename(columns=rename_dict_itens)


In [ ]:
chave_pedidos = 'ID_ITEM' if 'ID_ITEM' in df_pedidos_limpo.columns else 'ITEM'
chave_itens = chave_item_itens

print(f"  Chave em PEDIDOS: {chave_pedidos}")
print(f"  Chave em ITENS: {chave_itens}")

# Verificar se as chaves existem
assert chave_pedidos in df_pedidos_limpo.columns, f"Chave {chave_pedidos} não encontrada em PEDIDOS"
assert chave_itens in df_itens_limpo.columns, f"Chave {chave_itens} não encontrada em ITENS"

# Verificar valores únicos nas chaves
print(f"  Valores únicos em PEDIDOS[{chave_pedidos}]: {df_pedidos_limpo[chave_pedidos].nunique():,}")
print(f"  Valores únicos em ITENS[{chave_itens}]: {df_itens_limpo[chave_itens].nunique():,}")


  Chave em PEDIDOS: ID_ITEM
  Chave em ITENS: ID_ITEM
  Valores únicos em PEDIDOS[ID_ITEM]: 29,740
  Valores únicos em ITENS[ID_ITEM]: 69,774


In [ ]:
df_comparacao = df_pedidos_limpo.merge(
    df_itens_limpo,
    left_on=chave_pedidos,
    right_on=chave_itens,
    how='inner'
)

print(f"  ✓ Merge concluído!")
print(f"  Registros: {len(df_comparacao):,}")
print(f"  Colunas: {len(df_comparacao.columns)}")


  ✓ Merge concluído!
  Registros: 289,968
  Colunas: 150


In [ ]:
if len(df_comparacao.columns) != len(set(df_comparacao.columns)):
    duplicadas = [col for col in df_comparacao.columns if list(df_comparacao.columns).count(col) > 1]
    print(f"  ⚠️ ATENÇÃO: Colunas duplicadas no resultado: {set(duplicadas)}")
    # Remover duplicatas
    df_comparacao = df_comparacao.loc[:, ~df_comparacao.columns.duplicated()]
    print(f"  ✓ Duplicatas removidas. Colunas restantes: {len(df_comparacao.columns)}")
else:
    print(f"  ✓ Nenhuma duplicata de coluna encontrada")

# Listar colunas pareadas para comparação
print("\n6. Colunas pareadas para comparação:")
colunas_pedido = [col for col in df_comparacao.columns if col.endswith('_PEDIDO')]
colunas_item = [col for col in df_comparacao.columns if col.endswith('_ITEM')]

print(f"  Colunas de PEDIDOS: {len(colunas_pedido)}")
print(f"  Colunas de ITENS: {len(colunas_item)}")

# Encontrar pares
pares = []
for col_pedido in colunas_pedido:
    base = col_pedido.replace('_PEDIDO', '')
    col_item = f"{base}_ITEM"
    if col_item in colunas_item:
        pares.append((base, col_pedido, col_item))

print(f"  Pares encontrados para comparação: {len(pares)}")
print("\n  Exemplos de pares:")
for base, col_pedido, col_item in pares[:5]:
    print(f"    {base:30s}: {col_pedido:40s} ↔ {col_item:40s}")

# 7. Mostrar amostra do resultado
print("\n7. Amostra do resultado:")
colunas_mostrar = ['ID_PEDIDO', 'ID_ITEM'] if 'ID_PEDIDO' in df_comparacao.columns else ['ID_ITEM']
colunas_mostrar += [par[1] for par in pares[:3]]  # Primeiros 3 pares (PEDIDO)
colunas_mostrar += [par[2] for par in pares[:3]]  # Primeiros 3 pares (ITEM)
colunas_mostrar = [col for col in colunas_mostrar if col in df_comparacao.columns]

print(df_comparacao[colunas_mostrar].head())


  ✓ Nenhuma duplicata de coluna encontrada

6. Colunas pareadas para comparação:
  Colunas de PEDIDOS: 75
  Colunas de ITENS: 75
  Pares encontrados para comparação: 72

  Exemplos de pares:
    ID                            : ID_PEDIDO                                ↔ ID_ITEM                                 
    VL_LAP                        : VL_LAP_PEDIDO                            ↔ VL_LAP_ITEM                             
    VL_VOLUMEPALETEFECHADOM3      : VL_VOLUMEPALETEFECHADOM3_PEDIDO          ↔ VL_VOLUMEPALETEFECHADOM3_ITEM           
    VL_REFILOCOMPRIMENTO          : VL_REFILOCOMPRIMENTO_PEDIDO              ↔ VL_REFILOCOMPRIMENTO_ITEM               
    VL_COMPRIMENTOINTERNO         : VL_COMPRIMENTOINTERNO_PEDIDO             ↔ VL_COMPRIMENTOINTERNO_ITEM              

7. Amostra do resultado:
   ID_PEDIDO ID_ITEM  ID_PEDIDO  VL_LAP_PEDIDO  \
0   584184-8  658931   584184-8            0.0   
1   584184-9  658931   584184-9            0.0   
2  582746-13  635790  582746-13  

In [ ]:
# ============================================================
# PARTE 5: ANALISAR DIVERGÊNCIAS (VERSÃO CORRIGIDA)
# ============================================================

print("\n" + "=" * 80)
print("ANÁLISE DE DIVERGÊNCIAS POR COLUNA")
print("=" * 80)

resultados = []

# Iterar sobre os pares encontrados
for base, col_pedido, col_item in pares:
    
    # Remover registros onde ambos são nulos
    df_valido = df_comparacao[
        df_comparacao[col_pedido].notna() | df_comparacao[col_item].notna()
    ].copy()
    
    if len(df_valido) == 0:
        continue
    
    # Determinar tipo da coluna
    tipo_col = df_comparacao[col_pedido].dtype
    
    # CORREÇÃO: Identificar booleanos explicitamente
    is_boolean = (
        pd.api.types.is_bool_dtype(tipo_col) or 
        str(tipo_col) == 'boolean' or
        base.startswith('FL_') or
        base.startswith('BL_')
    )
    
    if is_boolean:
        # ========================================
        # TRATAMENTO PARA BOOLEANOS
        # ========================================
        
        # Converter para valores comparáveis (True/False/None)
        val_pedido = df_valido[col_pedido].fillna(False)
        val_item = df_valido[col_item].fillna(False)
        
        # Comparação exata
        divergencias = val_pedido != val_item
        
        n_divergencias = divergencias.sum()
        pct_divergencias = (n_divergencias / len(df_valido)) * 100
        
        # Calcular concordância
        concordancia = (val_pedido == val_item).sum()
        pct_concordancia = (concordancia / len(df_valido)) * 100
        
        resultado = {
            'Campo': base,
            'Coluna_Pedidos': col_pedido,
            'Coluna_Itens': col_item,
            'Tipo': 'Booleano',
            'Registros_válidos': len(df_valido),
            'Divergências': n_divergencias,
            'Pct_divergências': pct_divergencias,
            'Pct_concordância': pct_concordancia,
            'Diff_média': np.nan,
            'Diff_mediana': np.nan,
            'Diff_max': np.nan,
            'Diff_std': np.nan
        }
        
    elif pd.api.types.is_numeric_dtype(tipo_col):
        # ========================================
        # TRATAMENTO PARA NUMÉRICOS
        # ========================================
        
        # Comparação numérica com tolerância
        val_pedido = df_valido[col_pedido].fillna(0)
        val_item = df_valido[col_item].fillna(0)
        
        divergencias = ~np.isclose(
            val_pedido, 
            val_item,
            rtol=0.01,  # 1% de tolerância
            atol=0.01,
            equal_nan=True
        )
        
        n_divergencias = divergencias.sum()
        pct_divergencias = (n_divergencias / len(df_valido)) * 100
        
        # Calcular diferenças
        diff = val_pedido - val_item
        diff_abs = diff.abs()
        
        resultado = {
            'Campo': base,
            'Coluna_Pedidos': col_pedido,
            'Coluna_Itens': col_item,
            'Tipo': 'Numérico',
            'Registros_válidos': len(df_valido),
            'Divergências': n_divergencias,
            'Pct_divergências': pct_divergencias,
            'Pct_concordância': 100 - pct_divergencias,
            'Diff_média': diff.mean() if n_divergencias > 0 else 0,
            'Diff_mediana': diff.median() if n_divergencias > 0 else 0,
            'Diff_max': diff_abs.max() if n_divergencias > 0 else 0,
            'Diff_std': diff.std() if n_divergencias > 0 else 0
        }
        
    else:
        # ========================================
        # TRATAMENTO PARA TEXTO/CATEGÓRICO
        # ========================================
        
        # Comparação exata para textos/categorias
        val_pedido = df_valido[col_pedido].astype(str)
        val_item = df_valido[col_item].astype(str)
        
        divergencias = val_pedido != val_item
        
        n_divergencias = divergencias.sum()
        pct_divergencias = (n_divergencias / len(df_valido)) * 100


ANÁLISE DE DIVERGÊNCIAS POR COLUNA


In [ ]:
# Verificar match
col_estadoft = 'ID_ESTADOFT_DETEC' if 'ID_ESTADOFT_DETEC' in df_final.columns else 'ESTADOFT_DETEC'
if col_estadoft in df_final.columns:
    match = df_final[col_estadoft].notna().sum()
    print(f"   Match rate: {match:,} ({match/len(df_final)*100:.2f}%)")

# 4.3 MERGE COM MAQUINA
print("\n4.3 Merge com MAQUINA...")
chave_maquina = 'CD_MAQUINA' if 'CD_MAQUINA' in df_final.columns else 'MAQUINA'

df_final = df_final.merge(
    df_maquina_prep,
    on=chave_maquina,
    how='left',
    suffixes=('', '_DUP_MAQUINA')
)
print(f"   Após merge: {len(df_final):,} registros")


4.3 Merge com MAQUINA...
   Após merge: 1,604,242 registros


In [ ]:
# Verificar match
col_tipo_maq = 'CD_TIPO_MAQUINA' if 'CD_TIPO_MAQUINA' in df_final.columns else 'ID_TIPO_MAQUINA'
if col_tipo_maq in df_final.columns:
    match = df_final[col_tipo_maq].notna().sum()
    print(f"   Match rate: {match:,} ({match/len(df_final)*100:.2f}%)")

# 4.4 MERGE COM FACAS
print("\n4.4 Merge com FACAS...")
chave_faca = 'CD_FACA' if 'CD_FACA' in df_final.columns else 'FACA'
chave_codfaca = 'CD_CODFACA' if 'CD_CODFACA' in df_facas_prep.columns else 'CODFACA'

df_final = df_final.merge(
    df_facas_prep,
    left_on=chave_faca,
    right_on=chave_codfaca,
    how='left',
    suffixes=('', '_DUP_FACAS')
)
print(f"   Após merge: {len(df_final):,} registros")

   Match rate: 1,586,195 (98.88%)

4.4 Merge com FACAS...
   Após merge: 1,604,242 registros


In [ ]:
print("\n4.5 Merge com PARADAS...")
chave_parada_tarefa = 'CD_CODIGOPARADAOUCONV' if 'CD_CODIGOPARADAOUCONV' in df_final.columns else 'CODIGOPARADAOUCONV'
chave_parada_tab = 'CD_PARADA' if 'CD_PARADA' in df_paradas_prep.columns else 'PARADA'

df_final = df_final.merge(
    df_paradas_prep,
    left_on=chave_parada_tarefa,
    right_on=chave_parada_tab,
    how='left',
    suffixes=('', '_DUP_PARADAS')
)
print(f"   Após merge: {len(df_final):,} registros")


4.5 Merge com PARADAS...
   Após merge: 1,604,242 registros


In [ ]:
df_final[(df_final['CD_CODIGOPARADAOUCONV'] == '0') & (df_final['ID_PEDIDO'] == '-1') & (df_final['CD_OP'] == '-1')].sort_values('ID_ITEM')

,CD_MAQUINA,FL_FLAGPARADA,CD_CODIGOPARADAOUCONV,CD_TURMA,CD_OP,ID_PEDIDO,ID_ITEM,FL_REPROGRAMACAO,VL_ARRANJO,VL_GRAMATURA,QT_QUANTIDADEPROGRAMADA,QT_CHAPASALIMENTADAS,QT_QUANTIDADEPRODUZIDA,QT_QUANTIDADEAJUSTE,VL_DURACAOPREVISTA,DT_INICIO,DT_FIM,DT_DIADATURMA,ID_IDCLIENTE,ID_USUARIO,CD_ORIGEMREGISTRO,TX_DESCORIGEMREGISTRO,FL_SKIPFEED,CD_OPONDULADA,VL_DURACAO,CD_FACA,TX_DESCRSTATUSPEDIDO,VL_GRAMATURA_DUP_PEDIDOS,VL_LARGURAINTERNA,VL_COMPRIMENTOINTERNO,VL_ALTURAINTERNA,VL_LARGPECA,VL_COMPPECA,TX_DESCRTIPODOPEDIDO,TX_TEXTOESTADOFT_DETEC,TX_TEXTOSTATUSFT,CD_TIPOABNT,FL_REFILADO,FL_RESINAINTERNA,FL_PALETIZADO,QT_PECASPORPACOTE,QT_PECASPORPALETE,QT_PACOTESPORPALETE,VL_AREABRUTAPECA,VL_AREALIQUIDAPECA,VL_AREABRUTACHAPA,VL_AREALIQUIDACHAPA,VL_PESOCAIXA,CD_TIPO_MAQUINA,QT_NRDECORES,ID_IDGRUPOMAQUINA,CD_CODFACA,ST_STATUS_FACA,VL_COMPRIMENTO_LAMINA,FL_DESATIVADOSN,CD_PARADA,TX_DESCRICAO_PARADA,FL_PARADA_CONVERSAO,FL_DESATIVADA,FL_FLAGEXTERNA
743440,ESTEIRA2,True,0,B,-1,-1,-1,False,0.0,0.0,0,0,0,0,0.0,2025-05-04,2025-05-04,2025-05-04,-1,445,4,Interface com esteira de paletização,False,-1,37.0,-1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,NaN,NaN,NaN,NaN,NaN,6,0,20,NaN,NaN,NaN,<NA>,NaN,NaN,<NA>,<NA>,<NA>
781432,ESTEIRA2,True,0,C,-1,-1,-1,False,0.0,0.0,0,0,0,0,0.0,2023-08-20,2023-08-20,2023-08-19,-1,445,-1,Antigo não classificado,False,-1,2.0,-1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,NaN,NaN,NaN,NaN,NaN,6,0,20,NaN,NaN,NaN,<NA>,NaN,NaN,<NA>,<NA>,<NA>
781433,ESTEIRA2,True,0,C,-1,-1,-1,False,0.0,0.0,0,0,0,0,0.0,2023-08-20,2023-08-20,2023-08-19,-1,445,-1,Antigo não classificado,False,-1,3.0,-1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,NaN,NaN,NaN,NaN,NaN,6,0,20,NaN,NaN,NaN,<NA>,NaN,NaN,<NA>,<NA>,<NA>
781434,ESTEIRA2,True,0,C,-1,-1,-1,False,0.0,0.0,0,0,0,0,0.0,2023-08-20,2023-08-20,2023-08-19,-1,445,-1,Antigo não classificado,False,-1,3.0,-1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,NaN,NaN,NaN,NaN,NaN,6,0,20,NaN,NaN,NaN,<NA>,NaN,NaN,<NA>,<NA>,<NA>
781435,ESTEIRA2,True,0,C,-1,-1,-1,False,0.0,0.0,0,0,0,0,0.0,2023-08-20,2023-08-20,2023-08-19,-1,445,-1,Antigo não classificado,False,-1,3.0,-1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,NaN,NaN,NaN,NaN,NaN,6,0,20,NaN,NaN,NaN,<NA>,NaN,NaN,<NA>,<NA>,<NA>
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
760383,ESTEIRA,True,0,C,-1,-1,-1,False,0.0,0.0,0,0,0,0,0.0,2025-08-06,2025-08-06,2025-08-06,-1,445,4,Interface com esteira de paletização,False,-1,12.0,-1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,NaN,NaN,NaN,NaN,NaN,6,0,20,NaN,NaN,NaN,<NA>,NaN,NaN,<NA>,<NA>,<NA>
760384,ESTEIRA,True,0,C,-1,-1,-1,False,0.0,0.0,0,0,0,0,0.0,2025-08-06,2025-08-06,2025-08-06,-1,445,4,Interface com esteira de paletização,False,-1,7.0,-1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,NaN,NaN,NaN,NaN,NaN,6,0,20,NaN,NaN,NaN,<NA>,NaN,NaN,<NA>,<NA>,<NA>
760385,ESTEIRA,True,0,C,-1,-1,-1,False,0.0,0.0,0,0,0,0,0.0,2025-08-06,2025-08-06,2025-08-06,-1,445,4,Interface com esteira de paletização,False,-1,18.0,-1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,NaN,NaN,NaN,NaN,NaN,6,0,20,NaN,NaN,NaN,<NA>,NaN,NaN,<NA>,<NA>,<NA>
760413,ESTEIRA2,True,0,B,-1,-1,-1,False,0.0,0.0,0,0,0,0,0.0,2023-08-21,2023-08-21,2023-08-21,-1,445,-1,Antigo não classificado,False,-1,3.0,-1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,NaN,NaN,NaN,NaN,NaN,6,0,20,NaN,NaN,NaN,<NA>,NaN,NaN,<NA>,<NA>,<NA>


In [ ]:
df_final = df_final[df_final['CD_MAQUINA']!='unitizadora1']

In [ ]:
df_final[(df_final['CD_CODIGOPARADAOUCONV'] == '0') ].sort_values('ID_ITEM')

,CD_MAQUINA,FL_FLAGPARADA,CD_CODIGOPARADAOUCONV,CD_TURMA,CD_OP,ID_PEDIDO,ID_ITEM,FL_REPROGRAMACAO,VL_ARRANJO,VL_GRAMATURA,QT_QUANTIDADEPROGRAMADA,QT_CHAPASALIMENTADAS,QT_QUANTIDADEPRODUZIDA,QT_QUANTIDADEAJUSTE,VL_DURACAOPREVISTA,DT_INICIO,DT_FIM,DT_DIADATURMA,ID_IDCLIENTE,ID_USUARIO,CD_ORIGEMREGISTRO,TX_DESCORIGEMREGISTRO,FL_SKIPFEED,CD_OPONDULADA,VL_DURACAO,CD_FACA,TX_DESCRSTATUSPEDIDO,VL_GRAMATURA_DUP_PEDIDOS,VL_LARGURAINTERNA,VL_COMPRIMENTOINTERNO,VL_ALTURAINTERNA,VL_LARGPECA,VL_COMPPECA,TX_DESCRTIPODOPEDIDO,TX_TEXTOESTADOFT_DETEC,TX_TEXTOSTATUSFT,CD_TIPOABNT,FL_REFILADO,FL_RESINAINTERNA,FL_PALETIZADO,QT_PECASPORPACOTE,QT_PECASPORPALETE,QT_PACOTESPORPALETE,VL_AREABRUTAPECA,VL_AREALIQUIDAPECA,VL_AREABRUTACHAPA,VL_AREALIQUIDACHAPA,VL_PESOCAIXA,CD_TIPO_MAQUINA,QT_NRDECORES,ID_IDGRUPOMAQUINA,CD_CODFACA,ST_STATUS_FACA,VL_COMPRIMENTO_LAMINA,FL_DESATIVADOSN,CD_PARADA,TX_DESCRICAO_PARADA,FL_PARADA_CONVERSAO,FL_DESATIVADA,FL_FLAGEXTERNA
743440,ESTEIRA2,True,0,B,-1,-1,-1,False,0.0,0.0,0,0,0,0,0.0,2025-05-04,2025-05-04,2025-05-04,-1,445,4,Interface com esteira de paletização,False,-1,37.0,-1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,NaN,NaN,NaN,NaN,NaN,6,0,20,NaN,NaN,NaN,<NA>,NaN,NaN,<NA>,<NA>,<NA>
781432,ESTEIRA2,True,0,C,-1,-1,-1,False,0.0,0.0,0,0,0,0,0.0,2023-08-20,2023-08-20,2023-08-19,-1,445,-1,Antigo não classificado,False,-1,2.0,-1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,NaN,NaN,NaN,NaN,NaN,6,0,20,NaN,NaN,NaN,<NA>,NaN,NaN,<NA>,<NA>,<NA>
781433,ESTEIRA2,True,0,C,-1,-1,-1,False,0.0,0.0,0,0,0,0,0.0,2023-08-20,2023-08-20,2023-08-19,-1,445,-1,Antigo não classificado,False,-1,3.0,-1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,NaN,NaN,NaN,NaN,NaN,6,0,20,NaN,NaN,NaN,<NA>,NaN,NaN,<NA>,<NA>,<NA>
781434,ESTEIRA2,True,0,C,-1,-1,-1,False,0.0,0.0,0,0,0,0,0.0,2023-08-20,2023-08-20,2023-08-19,-1,445,-1,Antigo não classificado,False,-1,3.0,-1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,NaN,NaN,NaN,NaN,NaN,6,0,20,NaN,NaN,NaN,<NA>,NaN,NaN,<NA>,<NA>,<NA>
781435,ESTEIRA2,True,0,C,-1,-1,-1,False,0.0,0.0,0,0,0,0,0.0,2023-08-20,2023-08-20,2023-08-19,-1,445,-1,Antigo não classificado,False,-1,3.0,-1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,NaN,NaN,NaN,NaN,NaN,6,0,20,NaN,NaN,NaN,<NA>,NaN,NaN,<NA>,<NA>,<NA>
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
760385,ESTEIRA,True,0,C,-1,-1,-1,False,0.0,0.0,0,0,0,0,0.0,2025-08-06,2025-08-06,2025-08-06,-1,445,4,Interface com esteira de paletização,False,-1,18.0,-1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,NaN,NaN,NaN,NaN,NaN,6,0,20,NaN,NaN,NaN,<NA>,NaN,NaN,<NA>,<NA>,<NA>
760386,ESTEIRA,True,0,C,-1,-1,-1,False,0.0,0.0,0,0,0,0,0.0,2025-08-06,2025-08-06,2025-08-06,-1,445,4,Interface com esteira de paletização,False,-1,14.0,-1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,NaN,NaN,NaN,NaN,NaN,6,0,20,NaN,NaN,NaN,<NA>,NaN,NaN,<NA>,<NA>,<NA>
760377,ESTEIRA,True,0,B,-1,-1,-1,False,0.0,0.0,0,0,0,0,0.0,2025-08-06,2025-08-06,2025-08-06,-1,445,4,Interface com esteira de paletização,False,-1,5.0,-1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,NaN,NaN,NaN,NaN,NaN,6,0,20,NaN,NaN,NaN,<NA>,NaN,NaN,<NA>,<NA>,<NA>
793962,ESTEIRA,True,0,B,-1,-1,-1,False,0.0,0.0,0,0,0,0,0.0,2025-03-08,2025-03-08,2025-03-08,-1,445,-1,Antigo não classificado,False,-1,5.0,-1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,NaN,NaN,NaN,NaN,NaN,6,0,20,NaN,NaN,NaN,<NA>,NaN,NaN,<NA>,<NA>,<NA>


In [ ]:
df_final[(df_final['CD_CODIGOPARADAOUCONV'] != '-1')  & (df_final['CD_OP'] == '-1') ].shape[0]

47710

In [ ]:
df_final[(df_final['CD_CODIGOPARADAOUCONV'] != '-1')  & (df_final['CD_OP'] == '-1') ].shape[0] / df_final[(df_final['CD_CODIGOPARADAOUCONV'] != '-1')  ].shape[0]*100

3.7660546729152706

três casos para quando temos CODIGOPARADAOUCONV:
1. O primeiro caso em que  96% dos dados que possuem parada, não havia ID_ITEM e ID_PEDIDO, mas consegui identificar que havia CD_OP, como CD_OP é formado por ID_ITEM e ID_PEDIDO, consegui fazer o inverso para tratar
2 . Mas há outro caso em que CD_OP quando é parada não existe e não consegui identificar algum padrão que justifique esse comportamento, já associei com as outras tables e não encontrei.
3. Por ultimo,  o caso de quando CODIGOPARADAOUCONV é 0, que não há um correspondente em paradas..

In [ ]:
df_final

In [ ]:
df_final[(df_final['CD_CODIGOPARADAOUCONV'] != '-1') & (df_final['ID_PEDIDO'] == '-1')].shape[0]/df_final.shape[0]*100

4.095392091716836

In [ ]:
df_final[df_final['CD_MAQUINA'] == 'ESTEIRA'].TX_DESCRICAO_PARADA.value_counts()

TX_DESCRICAO_PARADA
NÃO HOUVE PRODUÇÃO NA DATA        4534
MANUTENÇÃO OPERACIONAL            3637
TROCA DE TURNO - AUT.              310
MANUTENÇÃO CORRETIVA MECÂNICA      225
MANUTENÇÃO CORRETIVA ELÉTRICA      179
PALETIZADORA CONGESTIONADA         157
AJUSTE                              90
FALTA DE PALETES                    66
MANUTENÇÃO PREVENTIVA               65
MANUTENÇÃO PREVENTIVA OPERAÇÃO      20
SISTEMA NÃO DISPONÍVEL              13
FALTA DE PROGRAMAÇÃO                11
FALHA OPERACIONAL                    9
MANUTENÇÃO CORRETI AMARRADEIRA       6
CURSOS/TREINAMENTOS/REUNIÕES         2
PROB AMARRADEIRA OPERACIONAL         1
FALTA DE ENERGIA / AR / ÁGUA         1
CRAVAMENTO NA ALIMENTAÇÃO            1
FALTA DE PESSOAL                     1
REFEIÇÃO                             1
FALTA DE INSUMOS                     1
Name: count, dtype: int64

In [ ]:
df_final[df_final['CD_MAQUINA'] == 'ESTEIRA2'].TX_DESCRICAO_PARADA.value_counts()

TX_DESCRICAO_PARADA
MANUTENÇÃO OPERACIONAL            3424
NÃO HOUVE PRODUÇÃO NA DATA        1057
MANUTENÇÃO CORRETIVA MECÂNICA      260
MANUTENÇÃO CORRETIVA ELÉTRICA      230
TROCA DE TURNO - AUT.              212
AJUSTE                             109
PALETIZADORA CONGESTIONADA          97
MANUTENÇÃO PREVENTIVA               39
SISTEMA NÃO DISPONÍVEL              18
MANUTENÇÃO PREVENTIVA OPERAÇÃO      10
FALTA DE PALETES                     5
FALHA OPERACIONAL                    4
FALTA DE ENERGIA / AR / ÁGUA         4
MANUTENÇÃO CORRETI AMARRADEIRA       3
LIMPEZA GERAL                        3
FALTA DE PROGRAMAÇÃO                 3
CURSOS/TREINAMENTOS/REUNIÕES         2
Name: count, dtype: int64

In [ ]:
df_final[df_final['CD_MAQUINA'] == 'unitizadora1'].TX_DESCRICAO_PARADA.value_counts()

TX_DESCRICAO_PARADA
FALTA DE PROGRAMAÇÃO    18047
Name: count, dtype: int64

In [ ]:
# Grupo 1: PRODUÇÃO (tem ITEM e PEDIDO)
df_producao = df_final[
    (df_final['ID_PEDIDO'] != '-1') & 
    (df_final['ID_ITEM'] != '-1')
].copy()

print(f"Produção: {len(df_producao):,} registros")
print(f"  - Com parada (conversão/setup): {(df_producao['FL_FLAGPARADA'] == True).sum():,}")
print(f"  - Produção normal: {(df_producao['FL_FLAGPARADA'] == False).sum():,}")

# Grupo 2: PARADAS SEM PRODUÇÃO (máquina parada, sem item)
df_paradas_gerais = df_final[
    (df_final['ID_PEDIDO'] == '-1') & 
    (df_final['ID_ITEM'] == '-1') &
    (df_final['FL_FLAGPARADA'] == True)
].copy()

print(f"\nParadas gerais: {len(df_paradas_gerais):,} registros")

# Grupo 3: REGISTROS INVÁLIDOS/INCOMPLETOS (investigar)
df_invalidos = df_final[
    ~(
        ((df_final['ID_PEDIDO'] != '-1') & (df_final['ID_ITEM'] != '-1')) |
        ((df_final['ID_PEDIDO'] == '-1') & (df_final['ID_ITEM'] == '-1') & (df_final['FL_FLAGPARADA'] == True))
    )
].copy()

print(f"\nInválidos (verificar): {len(df_invalidos):,} registros")

Produção: 1,538,485 registros
  - Com parada (conversão/setup): 1,219,133
  - Produção normal: 319,352

Paradas gerais: 47,710 registros

Inválidos (verificar): 0 registros
